In [139]:
pip install scikit-learn nltk 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [140]:
import pandas as pd
import yaml
from sqlalchemy import create_engine
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from nltk.corpus import stopwords

## Database Connection

In [141]:
with open('config.yml', 'r') as f: #Abrir el archivo en modo de  lectura
    config = yaml.safe_load(f) # Crear un diccionario con lo que hay en el archivo
    config_db_etl = config['bodega'] #Obtener solo la configuración de la bodega
    config_db = config["fuente"] #Obtener solo la configuración de la bodega

In [142]:
# Construct the database URL
url_db_etl = (f"{config_db_etl['driver']}://{config_db_etl['user']}:{config_db_etl['password']}@{config_db_etl['host']}:"
           f"{config_db_etl['port']}/{config_db_etl['db']}")
url_db = (f"{config_db['driver']}://{config_db['user']}:{config_db['password']}@{config_db['host']}:"
           f"{config_db['port']}/{config_db['db']}")

In [143]:
# Create the SQLAlchemy Engine
etl_conn = create_engine(url_db_etl)
olap_conn = create_engine(url_db)

## Extraction

In [144]:
descripcion_novedad = pd.read_sql_table('dim_descripcion_novedad', etl_conn) 
novedad_servicio = pd.read_sql_table("mensajeria_novedadesservicio",url_db)

In [145]:
novedad_servicio.head(5)

,id,fecha_novedad,tipo_novedad_id,descripcion,servicio_id,es_prueba,mensajero_id
0,4,2023-11-30 05:00:00+00:00,1,A,51,True,7
1,5,2023-11-30 05:00:00+00:00,1,Halo,51,True,7
2,6,2023-11-30 05:00:00+00:00,1,A,51,True,7
3,7,2023-11-30 05:00:00+00:00,1,B,51,True,7
4,8,2023-11-30 05:00:00+00:00,1,A,51,True,7


## Transformation

In [146]:
num_clusters = 10  # Número original de grupos
extra_group_name = "Grupo Especial"  # Nombre para el nuevo grupo

# Definir una lista de stopwords en español
stopwords_spanish = [
    'yo', 'tú', 'él', 'ella', 'usted', 'nosotros', 'vosotros', 'ellos',
    'la', 'el', 'al', 'los', 'las', 'de', 'y', 'en', 'que', 'a', 'es', 'un', 
    'una', 'por', 'para', 'con', 'no', 'si', 'se', 'me', 'te', 'le', 
    'lo', 'la', 'sus', 'mi', 'su', 'esto', 'eso', 'aquello', 'este', 
    'esa', 'aquella', 'ser', 'haber', 'estar', 'tener', 'hacer', 
    'poder', 'decir', 'ir', 'ver', 'dar', 'saber', 'querer', 'llegar',
    'pasar', 'deber', 'poner', 'parecer', 'quedar', 'creer', 'hablar', 
    'llevar', 'dejar', 'seguir', 'encontrar', 'llamar', 'venir', 
    'pensar', 'salir', 'volver', 'tomar', 'conocer', 'vivir', 
    'sentir', 'tratar', 'mirar', 'contar', 'empezar', 'esperar',
    'buscar', 'existir', 'entrar', 'trabajar', 'escribir', 
    'perder', 'producir', 'ocurrir', 'entender', 'poder', 'probar',
    'saber', 'así', 'mientras', 'cualquier', 'algunos', 'muchos', 'cualquier cosa', 
    'todos', 'cada', 'pocos', 'nada', 'todo', 'b3', 'b66', 'ya', 'qué'
]

# Separar documentos especiales y normales
special_documents = []
normal_documents = []

for doc in novedad_servicio['descripcion']:
    # Verificar si el documento es una letra, una sola palabra o vacío
    if len(doc.strip()) == 1 or doc.strip() == "":
        special_documents.append(doc)
    elif len(doc.strip().split()) == 1:
        special_documents.append(doc)
    else:
        normal_documents.append(doc)

# Combinar los documentos normales y especiales
documents = normal_documents + special_documents

# Crear el vectorizador TF-IDF
vectorizer = TfidfVectorizer(stop_words=stopwords_spanish)
X = vectorizer.fit_transform(documents)

# Instanciar y ajustar el modelo K-Means
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(X)

# Obtener las etiquetas de los clusters
labels = kmeans.labels_

# Agregar etiquetas al DataFrame original
novedad_servicio['grupo'] = None  # Inicializar una nueva columna para los grupos

# Asignar las etiquetas a las descripciones en el DataFrame
for i, doc in enumerate(novedad_servicio['descripcion']):
    if len(doc.strip()) == 1 or doc.strip() == "" or (len(doc.strip().split()) == 1):
        novedad_servicio.at[i, 'grupo'] = 10  # Asignar al grupo especial
    else:
        novedad_servicio.at[i, 'grupo'] = labels[i - len(special_documents)]  # Ajustar el índice para documentos normales

c:\Users\nicol\Documents\Univalle\semestre_7_8\DataScience\proyecto\venv\Lib\site-packages\sklearn\feature_extraction\text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['cosa'] not in stop_words.
  warnings.warn(


In [154]:
# Mostrar el DataFrame con las descripciones y su clasificación
novedad_servicio.head(5)

,id,fecha_novedad,tipo_novedad_id,descripcion,servicio_id,es_prueba,mensajero_id,grupo
0,4,2023-11-30 05:00:00+00:00,1,A,51,True,7,10
1,5,2023-11-30 05:00:00+00:00,1,Halo,51,True,7,10
2,6,2023-11-30 05:00:00+00:00,1,A,51,True,7,10
3,7,2023-11-30 05:00:00+00:00,1,B,51,True,7,10
4,8,2023-11-30 05:00:00+00:00,1,A,51,True,7,10


In [148]:
hecho_novedad_servicio = novedad_servicio[["id", "descripcion", "grupo"]]
hecho_novedad_servicio = pd.merge(hecho_novedad_servicio, descripcion_novedad[["key_descripcion_novedad"]], left_on="grupo", right_on="key_descripcion_novedad", how="left")
hecho_novedad_servicio.head(30)

,id,descripcion,grupo,key_descripcion_novedad
0,4,A,10,10
1,5,Halo,10,10
2,6,A,10,10
3,7,B,10,10
4,8,A,10,10
5,9,V,10,10
6,10,V,10,10
7,11,C,10,10
8,12,B,10,10
9,15,Me pinché,2,2


In [149]:
melo = novedad_servicio.groupby("grupo").size().reset_index(name="cantidad_por_descripcion_novedad")
melo
nov = novedad_servicio[novedad_servicio["grupo"] == 6]
nov.head(10)

,id,fecha_novedad,tipo_novedad_id,descripcion,servicio_id,es_prueba,mensajero_id,grupo
143,149,2024-02-15 19:03:48.145156+00:00,1,El cliente está llamando al asesor,1272,True,38,6
226,231,2024-02-23 15:30:48.467831+00:00,1,En espera que el cliente llegue,2040,True,22,6
227,232,2024-02-23 15:30:50.523278+00:00,1,En espera que el cliente llegue,2040,True,22,6
228,233,2024-02-23 15:55:58.127991+00:00,1,No deja cargar foto y comentario del domi muni...,2059,True,25,6
229,234,2024-02-23 15:55:59.073888+00:00,1,No deja cargar foto y comentario del domi muni...,2059,True,25,6
230,235,2024-02-23 16:12:19.712056+00:00,1,Se entrega a las 1058 AM se reporta a esta hor...,2055,True,15,6
231,236,2024-02-23 16:12:21.262857+00:00,1,Se entrega a las 1058 AM se reporta a esta hor...,2055,True,15,6
232,237,2024-02-23 17:16:17.892128+00:00,1,No actualiza y se reportará a los grupos de me...,2084,True,31,6
233,238,2024-02-23 17:17:17.178963+00:00,1,No actualiza y se reportará por los grupos de ...,2068,True,31,6
235,240,2024-02-23 17:29:11.854954+00:00,1,Se entrega a las 12:09 aplicación fallando no ...,2075,True,15,6


In [150]:
descripcion_novedad.head(10)

,key_descripcion_novedad,grupo,terminos_clave,descripcion
0,0,Grupo 0,"está, informa, listo, cerrado, servicio, aseso...",Estado del servicio y atención al cliente.
1,1,Grupo 1,"espera, están, sigo, entreguen, terminal, reci...",Coordinación de entregas y tiempos de espera.
2,2,Grupo 2,"pedido, despachen, esperando, espera, entregue...",Gestión de pedidos y despachos.
3,3,Grupo 3,"toma, compañero, otro, asunto, trasbordo, serv...",Interacción y colaboración entre compañeros.
4,4,Grupo 4,"servicio, camioneta, mal, programado, persona,...",Problemas y situaciones adversas en servicios.
5,5,Grupo 5,"compañero, otro, servicio, entrego, continúa, ...",Trabajo en equipo y continuidad en entregas.
6,6,Grupo 6,"enterado, pendiente, ok, recoger, salgo, aseso...",Seguimiento de acciones pendientes y recogida.
7,7,Grupo 7,"deja, repuesto, escritorio, yulieth, mañana, s...",Logística de repuestos y tareas administrativas.
8,8,Grupo 8,"esperando, despacho, bodega, entrega, reciban,...",Estado de alistamiento y procesos de entrega.
9,9,Grupo 9,"repuesto, entrega, hay, informa, encuentra, se...",Disponibilidad de productos y comunicación al ...


In [151]:
# Contar solicitudes por cada 'key_descripcion_novedad'
cantidad_por_descripcion_novedad = hecho_novedad_servicio.groupby("key_descripcion_novedad").size().reset_index(name="cantidad_por_descripcion_novedad")
# Hacer merge para agregar la columna 'cantidad_por_descripcion_novedad' al DataFrame original
hecho_novedad_servicio = hecho_novedad_servicio.merge(cantidad_por_descripcion_novedad, on="key_descripcion_novedad", how="left")
# Eliminar columnas innecesarias
hecho_novedad_servicio.drop(columns=["descripcion", "grupo"], inplace=True)
# Mostrar las primeras 10 filas del DataFrame resultante
hecho_novedad_servicio.head(10)

,id,key_descripcion_novedad,cantidad_por_descripcion_novedad
0,4,10,109
1,5,10,109
2,6,10,109
3,7,10,109
4,8,10,109
5,9,10,109
6,10,10,109
7,11,10,109
8,12,10,109
9,15,2,2128


In [152]:
print(len(hecho_novedad_servicio))

5208


## Loading

In [153]:
hecho_novedad_servicio.to_sql("hecho_novedad_servicio", etl_conn, if_exists="replace", index_label="key_novedad_servicio") 

208